In [1]:
import copy
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm

In [2]:
# random_seed = 0
# np.random.seed(random_seed)
# torch.manual_seed(random_seed)
# random.seed(random_seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
image_size = 32

In [32]:
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms
from numpy.random import RandomState
from torch.utils.data import Subset

from easyfsl.samplers import TaskSampler


batch_size = 128
n_workers = 1

cifar_data = CIFAR10(root="data", train=True, download=True,  transform= transforms.Compose([
            transforms.RandomResizedCrop(image_size),
            transforms.RandomHorizontalFlip(),
            
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
        ]))

cifar_data_val = CIFAR10(root='.',train=True, transform= transforms.Compose([
            # transforms.RandomResizedCrop(image_size),
            # transforms.RandomHorizontalFlip(),
            
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
        ]), download=True)

n_way = 2
n_shot = 1
n_query = 4
n_tasks_per_epoch = 5
n_validation_tasks = 200

def get_dataloader(dataset,dataset_val, n_workers, seed):
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 500))
    classes =  prng.permutation(np.arange(0,10))
    indx_train = np.concatenate([np.where(np.array(dataset.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    indx_val = np.concatenate([np.where(np.array(dataset_val.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])
    train_targets = np.array(dataset.targets)[indx_train]
    val_targets = np.array(dataset.targets)[indx_val]
    train_data = Subset(cifar_data, indx_train)
    print(len(train_data.dataset.targets))
    val_data = Subset(cifar_data_val, indx_val)
    train_data.get_labels = lambda: [
        t for t in train_targets
    ] 
    val_data.get_labels = lambda: [
        t for t in val_targets
    ] 
    train_sampler = TaskSampler(
        train_data, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_tasks_per_epoch
    )
    val_sampler = TaskSampler(
        val_data, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks
    )

    train_loader = DataLoader(
        train_data,
        batch_sampler=train_sampler,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=train_sampler.episodic_collate_fn,
    )
    
    val_loader = DataLoader(
        val_data,
        batch_sampler=val_sampler,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=val_sampler.episodic_collate_fn,
    )
    
    return train_loader, val_loader
    
random_seed = 0


Files already downloaded and verified
Files already downloaded and verified
50000
Train set size: 5
Validation set size: 200


In [42]:
from easyfsl.modules import resnet12, resnet50, 
from torchvision.models import resnet18

DEVICE = "cuda"

# model = resnet18(
#     pretrained=True, 
# )

# model.fc = nn.Sequential(
#     nn.Linear(512, 256), 
#     nn.ReLU(),
#     nn.Linear(256, 256),
#     nn.Linear(256, 10)
# )


model = resnet50()
model = model.to(DEVICE)

In [43]:
from easyfsl.methods import PrototypicalNetworks, LaplacianShot, MatchingNetworks, SimpleShot, TransductiveFinetuning, PTMAP, BDCSPN

few_shot_classifier = LaplacianShot(model).to(DEVICE)



In [50]:
from torch.optim import SGD, Optimizer, Adam
from torch.optim.lr_scheduler import MultiStepLR, CosineAnnealingLR
from torch.utils.tensorboard import SummaryWriter


LOSS_FUNCTION = nn.CrossEntropyLoss()

n_epochs = 20
scheduler_milestones = [120, 180]
scheduler_gamma = 0.1
learning_rate = 0.01
tb_logs_dir = Path(".")

# train_optimizer = SGD(
#     model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
# )

train_optimizer = Adam(
    model.parameters(), lr=learning_rate, 
)
train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)


# train_scheduler = CosineAnnealingLR(
#     train_optimizer,
#     T_max=100
#     # milestones=scheduler_milestones,
#     # gamma=scheduler_gamma,
# )

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

In [51]:
def training_epoch(
    model, data_loader: DataLoader, optimizer: Optimizer
):
    all_loss = []
    model.train()
    with tqdm(
        enumerate(data_loader), total=len(data_loader), desc="Training"
    ) as tqdm_train:
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            _,
        ) in tqdm_train:
            optimizer.zero_grad()
            model.process_support_set(
                support_images.to(DEVICE), support_labels.to(DEVICE)
            )
            classification_scores = model(query_images.to(DEVICE))

            loss = LOSS_FUNCTION(classification_scores, query_labels.to(DEVICE))
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [52]:
from easyfsl.utils import evaluate


best_state = model.state_dict()
best_validation_accuracy = 0.0
validation_frequency = 10

validation_accuracies = []
for seed in range(5):
    
    train_loader, val_loader = get_dataloader(cifar_data,cifar_data_val, n_workers, seed)
    print("Train set size:", len(train_loader))
    print("Validation set size:", len(val_loader))
    
    for epoch in range(n_epochs):
        print(f"Epoch {epoch}")
        average_loss = training_epoch(few_shot_classifier, train_loader, train_optimizer)

        # if epoch % validation_frequency == validation_frequency - 1:

            # model.eval()
            # validation_accuracy = evaluate(
            #     few_shot_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
            # )
            # # model.train()
            # if validation_accuracy > best_validation_accuracy:
            #     best_validation_accuracy = validation_accuracy
            #     best_state = copy.deepcopy(few_shot_classifier.state_dict())
            #     print(" found a new best model!")

            # tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

        tb_writer.add_scalar("Train/loss", average_loss, epoch)

        # Warn the scheduler that we did an epoch
        # so it knows when to decrease the learning rate
        train_scheduler.step()
        
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
    )
    validation_accuracies.append(validation_accuracy)
    
print(
        f"Accuracy: {100* np.array(validation_accuracies).mean():.3f}% \u00B1 {100*np.array(validation_accuracies).std():.3f}%"
    )

50000
Train set size: 5
Validation set size: 200
Epoch 0


Training: 100%|██████████| 5/5 [00:06<00:00,  1.27s/it, loss=0.788]

Epoch 1



Training: 100%|██████████| 5/5 [00:00<00:00, 15.32it/s, loss=0.863]

Epoch 2


...

Epoch 18



Training: 100%|██████████| 5/5 [00:00<00:00, 14.14it/s, loss=0.738]

Epoch 19



Validation: 100%|██████████| 200/200 [00:04<00:00, 48.56it/s, accuracy=0.534]


Accuracy: 52.037% ± 1.678%
